In [1]:
library(Seurat)
library(reticulate)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject



In [2]:
use_python("/data/chenyx/anaconda3/envs/PICASSO/bin")

In [3]:
sc <- import("scanpy")
sp <- import("scipy.sparse")

# load data

In [4]:
adata <- sc$read_h5ad("/nfs/public/cell_gpt_data/Intergation_Data/Lung/Lung_atlas_uniformed.h5ad")

In [5]:
expression_matrix = as.matrix(adata$X)
rownames(expression_matrix) <- row.names(adata$obs)
colnames(expression_matrix) <- row.names(adata$var)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 10.2 GiB”


In [6]:
dataobj <- CreateSeuratObject(counts = t(expression_matrix), meta.data = adata$obs, min.cells = 0,min.features = 0)
dataobj

An object of class Seurat 
42117 features across 32472 samples within 1 assay 
Active assay: RNA (42117 features, 0 variable features)

In [7]:
dataobj <- NormalizeData(dataobj)

In [8]:
dataobj <- dataobj[read.csv("gene_list.csv")$gene_name,]

In [9]:
dataobj

An object of class Seurat 
2000 features across 32472 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)

In [10]:
saveRDS(dataobj, "/nfs/public/cell_gpt_data/Intergation_Data/Lung/selected_gene.rds")

# FastMNN

In [11]:
library(SeuratWrappers)

In [ ]:
dataobj <- readRDS("/nfs/public/cell_gpt_data/Intergation_Data/Lung/selected_gene.rds")

In [12]:
start_time = Sys.time()
dataobj.fastmnn <- RunFastMNN(object.list = SplitObject(dataobj, split.by = "batch"))
end_time = Sys.time()

Computing 2000 integration features

No variable features found for object1 in the object.list. Running FindVariableFeatures ...

No variable features found for object2 in the object.list. Running FindVariableFeatures ...

No variable features found for object3 in the object.list. Running FindVariableFeatures ...

No variable features found for object4 in the object.list. Running FindVariableFeatures ...

No variable features found for object5 in the object.list. Running FindVariableFeatures ...

No variable features found for object6 in the object.list. Running FindVariableFeatures ...

No variable features found for object7 in the object.list. Running FindVariableFeatures ...

No variable features found for object8 in the object.list. Running FindVariableFeatures ...

No variable features found for object9 in the object.list. Running FindVariableFeatures ...

No variable features found for object10 in the object.list. Running FindVariableFeatures ...

No variable features found for o

In [13]:
end_time- start_time

Time difference of 1.247391 mins

In [14]:
saveRDS(dataobj.fastmnn, "/nfs/public/cell_gpt_data/Intergation_Data/Lung/Results/FastMNN_result.rds")

In [15]:
dataobj.fastmnn@active.assay <- "mnn.reconstructed"

In [16]:
dataobj.fastmnn

An object of class Seurat 
3650 features across 32472 samples within 2 assays 
Active assay: mnn.reconstructed (1650 features, 1650 variable features)
 1 other assay present: RNA
 1 dimensional reduction calculated: mnn

In [17]:
X = t(as.matrix(dataobj.fastmnn@assays$mnn.reconstructed@data))
var = VariableFeatures(dataobj.fastmnn)
obs = dataobj.fastmnn[[]]

In [18]:
adata_seurat <- sc$AnnData(
    X   = X,
    obs = obs,
    var = VariableFeatures(dataobj.fastmnn)
)
adata_seurat$X  <- sp$csc_matrix(adata_seurat$X)

In [19]:
colnames(adata_seurat$var) <- "gene_name"
adata_seurat

AnnData object with n_obs × n_vars = 32472 × 1650
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'gene_name'

In [20]:
adata_seurat$write_h5ad("/nfs/public/cell_gpt_data/Intergation_Data/Lung/Results/FastMNN_result.h5ad")

# Seurat integration

In [21]:
start_time = Sys.time()
dataobj.list <- SplitObject(dataobj, split.by = "batch")
anchors <- FindIntegrationAnchors(object.list = dataobj.list, anchor.features = read.csv("gene_list.csv")$gene_name)
dataobj.integrated <- IntegrateData(anchorset = anchors)
end_time = Sys.time()

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 6248 anchors

Filtering anchors

	Retained 3171 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 4750 anchors

Filtering anchors

	Retained 2513 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 5407 anchors

Filtering anchors

	Retained 2831 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 5758 anchors

Filtering anchors

	Retained 2669 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 6559 anchors

Filtering anchors

	Retained 2726 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 4867 anchors

Filtering anchors

	Retained 2630 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 6282 anchors

Filtering anchors

	Retained 1901 an

In [22]:
end_time- start_time

Time difference of 23.01893 mins

In [23]:
dataobj.integrated

An object of class Seurat 
4000 features across 32472 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 other assay present: RNA

In [24]:
saveRDS(dataobj.integrated, "/nfs/public/cell_gpt_data/Intergation_Data/Lung/Results/Seurat_result.rds")

In [25]:
X = t(as.matrix(dataobj.integrated@assays$integrated@data))
var = VariableFeatures(dataobj.integrated)
obs = dataobj.integrated[[]]

In [26]:
adata_seurat <- sc$AnnData(
    X   = X,
    obs = obs,
    var = VariableFeatures(dataobj.integrated)
)
adata_seurat$X  <- sp$csc_matrix(adata_seurat$X)
colnames(adata_seurat$var) <- "gene_name"
adata_seurat

AnnData object with n_obs × n_vars = 32472 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'gene_name'

In [27]:
adata_seurat$write_h5ad("/nfs/public/cell_gpt_data/Intergation_Data/Lung/Results/Seurat_result.h5ad")